# 100 Prepare input scenario database
In this notebook, we will prepare the input scenario database.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
AR6_FILTERS = {
    'IMP_marker':[
        'SP', 'Neg', 'Ren'
    ],
    #'Category':['C1', 'C2', 'C3']
}

<IPython.core.display.Javascript object>

In [3]:
import pyam
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from utils import calculate_netzero

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Read in the input dataframe
The file should be in the `data` folder and is called: `20220314_ar6emissions_harmonized_infilled.csv`

In [4]:
df_input = pyam.IamDataFrame(
    Path(
        '../data/20220314_ar6emissions_harmonized_infilled.csv'
    )
)

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../data/20220314_ar6emissions_harmonized_infilled.csv


<IPython.core.display.Javascript object>

For now, we will also add in the metadata that comes along with the AR6 scenario database.

In [5]:
df_input.load_meta(
    Path(
        '../data/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx'
    )
)

pyam.core - WARNING: Reading meta indicators for 1703 out of 1836 scenarios, ignoring 601 scenarios from file


<IPython.core.display.Javascript object>

## Data processing

Step 1: Filter out the necessary scenarios

In [6]:
df_proc = df_input.filter(
    **AR6_FILTERS
)

<IPython.core.display.Javascript object>

Step 2: Check if total CO2 emissions are reported.

In [7]:
df_proc.require_variable(
    'AR6 climate diagnostics|Infilled|Emissions|CO2'
)

/var/folders/yw/qzffs_jd1l3b0w2tk4cdqxj00000gn/T/ipykernel_56666/255023924.py:1: DeprecationWarning: This method is deprecated and will be removed in future versions. Use `df.require_data()` instead.
  df_proc.require_variable(
pyam.core - INFO: 3 scenarios do not include required variable `AR6 climate diagnostics|Infilled|Emissions|CO2`


,model,scenario
0,COFFEE 1.1,EN_NPi2020_400f_lowBECCS
1,REMIND-MAgPIE 2.1-4.2,SusDev_SDP-PkBudg1000
2,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_ HighRE_Budg900


<IPython.core.display.Javascript object>

Step 3: It appears not. So let us construct this variable.

In [8]:
df_proc.aggregate(
    variable='AR6 climate diagnostics|Infilled|Emissions|CO2',
    components=[
        'AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes',
        'AR6 climate diagnostics|Infilled|Emissions|CO2|AFOLU'
    ],
    append=True
)

<IPython.core.display.Javascript object>

Step 4: Assign the year of net zero CO2. It should already be present in the metadata, but it won't hurt to recompute it.

In [9]:
co2 = (
    df_proc
    .filter(
        variable='AR6 climate diagnostics|Infilled|Emissions|CO2'
    )
    .swap_time_for_year()
    .timeseries()
)

<IPython.core.display.Javascript object>

In [10]:
df_proc.set_meta(
    calculate_netzero(co2),
    'netzero|CO2'
)

<IPython.core.display.Javascript object>

## Save out the data

In [11]:
df_proc.to_csv(
    Path(
        '../data/100_scenarios.csv'
    )
)

<IPython.core.display.Javascript object>

In [12]:
df_proc.export_meta(
    Path(
        '../data/100_meta.xlsx'
    )
)

<IPython.core.display.Javascript object>